In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
try:
    from PyEMD import EEMD
except:
    !pip install EMD-signal
    from PyEMD import EEMD

from ta import add_all_ta_features

In [3]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl
from pytorch_lightning.metrics import F1
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.loggers import TensorBoardLogger

In [4]:
CURRENCY_LST = ['BTC']
PRICE_TYPE = 'close'
FREQUENCY = "D"
WINDOW_SIZE = 10
NEUTRAL_QUANTILE = 0.25
LOG_PRICE = True
INDICATORS = False
IMFS = True

In [5]:
N_CLASSES = 2 ###
TRAIN_PERCENTAGE, VAL_PERCENTAGE, TEST_PERCENTAGE = 0.90, 0.10, 0.10

In [6]:
LSTM_HIDDEN_SIZES = [128, 128, 128]

In [7]:
BATCH_SIZE= 32

In [8]:
def get_data(currency_lst,
             frequency, 
             window_size,
             neutral_quantile = 0.25,
             beg_date = pd.Timestamp(2013,1,1),
             end_date = pd.Timestamp.now(),
             log_price = True, 
             include_indicators = True,
             include_imfs = True):
        
        X, y, dfs = {}, {}, {}     
        
        for cur in currency_lst:
            df = pd.read_csv(f"../data/0_raw/Binance/{str.lower(cur)}_usdt_1d.csv", index_col=0).reset_index()   
            
            if include_indicators:
                df = add_all_ta_features(df, open="Open", high="High", low="Low", close="Close", volume="Volume", 
                                         fillna=True)
            else:
                df.drop("Volume", axis=1, inplace=True)
            
            df.Date = df.Date.apply(pd.Timestamp)
            df.sort_values("Date", ascending=True, inplace=True)
            df.set_index("Date", inplace=True)
            df.drop(["Timestamp", "Open", "High", "Low"], axis=1, inplace=True)
            df.rename(str.lower, axis=1, inplace=True)
            
            if log_price:
                df["close"] = df["close"].apply(np.log)
      
            if include_imfs:
                eemd = EEMD()
                imfs = eemd(df[PRICE_TYPE].values)
                imf_features = ["imf_"+str(i) for i in range(imfs.shape[0])]
                df = pd.concat((df, pd.DataFrame(imfs.T, columns=imf_features, index=df.index)), axis=1)
                
            price_diff = df["close"].diff().dropna()
            rolling_quantiles = price_diff.abs().rolling(window_size).quantile(neutral_quantile).dropna()
#             conditions = [(price_diff[window_size-1:] < 0) & (price_diff[window_size-1:].abs() > rolling_quantiles),
#                           (price_diff[window_size-1:] > 0) & (price_diff[window_size-1:].abs() > rolling_quantiles)]
            conditions = [(price_diff[window_size-1:] < 0),
                          (price_diff[window_size-1:] > 0)]

#             classes = [1,2] #1 is decrease, 2 is decrease, and 0 is neutral if none of conditions is met
            classes = [0,1]
    
            y = pd.Series(np.select(conditions, classes, default=0), index=price_diff[window_size-1:].index)
            df.insert(loc=0, column="change_dir", value=y)
            dfs[cur] = df
            
        min_dates = [df.index.min() for cur, df in dfs.items()]
        max_dates = [df.index.max() for cur, df in dfs.items()]
        beg_date = max([max(min_dates), beg_date])
        end_date = min([min(max_dates), end_date])
        common_range = pd.date_range(beg_date, end_date, freq=frequency)
        
        X = np.array([dfs[cur].drop("change_dir", axis=1).loc[common_range].values for cur in currency_lst])
        y = np.array([dfs[cur].loc[common_range, "change_dir"].values for cur in currency_lst])
        features = df.columns.tolist()
        
        return X, y, features, dfs

In [9]:
X, y, features, dfs = get_data(CURRENCY_LST,
                                 FREQUENCY, 
                                 WINDOW_SIZE,
                                 neutral_quantile = NEUTRAL_QUANTILE,
                                 log_price=LOG_PRICE,
                                 include_indicators = INDICATORS,
                                 include_imfs = IMFS
                                )

In [10]:
N_CURRENCIES = X.shape[0]
INPUT_FEATURE_SIZE = X.shape[-1]

In [11]:
(X.shape, y.shape)

((1, 1360, 10), (1, 1360))

In [27]:
X[0][0]

array([ 8.3628945 ,  0.04117395,  0.00983016, -0.01647546, -0.0388904 ,
       -0.05112657, -0.42991952,  0.46521155,  7.70351254,  8.75648574])

In [12]:
class TimeSeriesDataset(Dataset):
    def __init__(self, 
                 x: np.ndarray,
                 y: np.ndarray,
                 data_use_type,
                 train_percentage = TRAIN_PERCENTAGE,
                 val_percentage = VAL_PERCENTAGE,
                 test_percentage = TEST_PERCENTAGE,
                 seq_len = WINDOW_SIZE, 
                 ):
        
        self.x = torch.tensor(x).float()
        self.y = torch.tensor(y).long()
        self.seq_len = seq_len
        
        self.data_use_type = data_use_type
        
        #self.train_size = int(len(self.x) * train_percentage)
        self.val_size = int(len(self.x) * val_percentage)
        self.test_size = int(len(self.x) * test_percentage)
        self.train_size = len(self.x) - self.val_size - self.test_size 
        
        self.train_mean = self.x[:self.train_size].mean()
        self.train_std = self.x[:self.train_size].std()
        
        self.train_min = self.x[:self.train_size].min()
        self.train_max = self.x[:self.train_size].max()
    
    def __len__(self):
        
        if self.data_use_type == "train":
            return self.train_size - self.seq_len
        
        elif self.data_use_type == "val":
            return self.val_size 
        
        else:
            return self.test_size
        
    def __getitem__(self, index):
        
        if self.data_use_type =="val":
            index = self.train_size + index - self.seq_len
            
        elif self.data_use_type =="test":
            index = self.train_size + self.val_size + index - self.seq_len
        
        window = self.x[index:index+self.seq_len]
        window = (window -self.train_min) / (self.train_max - self.train_min)
#         window = (window -self.train_mean) / self.train_std
        
        price_change = self.y[index+self.seq_len]
        
#         window = self.x[index:index+self.seq_len]
#         window = (window -self.train_mean) / self.train_std 
        
#         price_change = self.x[index+self.seq_len] - self.x[index+self.seq_len-1]
#         price_change =  2 if price_change == 0 else 1 if price_change >0 else 0 #2 değişmemesi, 1 artış, 0 düşüş
#         #burada 2 hiç gelmiyor N_CLASSES = 2 dediğinde run edince patlardı
        
        return (window, price_change)

In [13]:
train_dataset, val_dataset, test_dataset = [TimeSeriesDataset(X[0], y[0], dtype) for dtype in ['train', 'val', 'test']]

In [14]:
test_dataset.__len__()

136

In [15]:
#calculate loss' weights
train_labels = [train_dataset[i][1].item() for i in range (train_dataset.__len__())]

cnt = Counter(train_labels)
samples_size = np.array([cnt[key] for key in  sorted(cnt.keys())])

loss_weights = (1 / samples_size) * sum(samples_size)/2
loss_weights, cnt

(array([1.05479452, 0.95061728]), Counter({0: 511, 1: 567}))

In [16]:
class LSTM_based_classification_model(pl.LightningModule):
    def __init__(self,
                 train_dataset = train_dataset,
                 val_dataset = val_dataset,
                 test_dataset = test_dataset,
                 weights = loss_weights,
                 num_classes = N_CLASSES,
                 window_size = WINDOW_SIZE,
                 input_size = INPUT_FEATURE_SIZE,
                 batch_size=8,
                 lstm_hidden_sizes = LSTM_HIDDEN_SIZES,
                 bidirectional = False,
                 ):
        
        super().__init__()
        self.num_classes = num_classes
        self.window_size = window_size
        self.input_size = input_size
        self.batch_size = batch_size
        
        self.lstm_hidden_sizes = lstm_hidden_sizes
        self.bidirectional = bidirectional 
        
#         self.stack_lstm = nn.LSTM(input_size = self.input_size, 
#                 hidden_size = self.lstm_hidden_size, 
#                 num_layers= self.lstm_stack_size,
#                 dropout = self.lstm_dropout, # sadece stack arasına koyuyor
#                 bidirectional = self.bidirectional, 
#                 batch_first=True,) 
        
        self.lstm_1 = nn.LSTM(input_size = self.input_size, num_layers=1, batch_first=True, hidden_size = self.lstm_hidden_sizes[0])
        self.batch_norm1 = nn.BatchNorm2d(num_features=self.lstm_hidden_sizes[0])
        
        self.lstm_2 = nn.LSTM(input_size = self.lstm_hidden_sizes[0], num_layers=1, batch_first=True, hidden_size = self.lstm_hidden_sizes[1])
        self.batch_norm2 = nn.BatchNorm2d(num_features=self.lstm_hidden_sizes[1])
        
        self.lstm_3 = nn.LSTM(input_size = self.lstm_hidden_sizes[1], num_layers=1, batch_first=True, hidden_size = self.lstm_hidden_sizes[2])
        self.batch_norm3 = nn.BatchNorm2d(num_features=self.lstm_hidden_sizes[2])
        
        self.dropout = nn.Dropout(0.5)
        
        self.linear1 = nn.Linear(self.lstm_hidden_sizes[2], int(self.lstm_hidden_sizes[2]/2))
        self.activation = nn.ReLU()
        
        self.output_layer = nn.Linear(int(self.lstm_hidden_sizes[2]/2), self.num_classes)
        
#         self.cross_entropy_loss = nn.CrossEntropyLoss(weight= torch.tensor(weights).float()) # loss weight
        self.cross_entropy_loss = nn.CrossEntropyLoss()
        
        self.f1_score = pl.metrics.F1(num_classes=self.num_classes, average="macro")
        self.accuracy_score = pl.metrics.Accuracy()
        
        self.train_dl = DataLoader(train_dataset, batch_size=self.batch_size, shuffle = True)
        self.val_dl = DataLoader(val_dataset, batch_size=self.batch_size*3)
        self.test_dl = DataLoader(test_dataset, batch_size=self.batch_size*3)
        
    def forward(self, x):
        batch_size = x.size()[0]
        
        x = x.view(batch_size, self.window_size, self.input_size) #(batch, window_len, feature_size)
        x, _  = self.lstm_1(x)
        x = self.dropout(x)

        x = x.reshape(x.size()[-1], batch_size, self.window_size) #(feature_size, batch, window_len)
        x = self.batch_norm1(x.unsqueeze(0))

        x = x.view(batch_size, self.window_size, x.size()[1])
        x, _  = self.lstm_2(x)
        x = self.dropout(x)
        
        x = x.reshape(x.size()[-1], batch_size, self.window_size) #(feature_size, batch, window_len)
        x = self.batch_norm2(x.unsqueeze(0))
        
        x = x.view(batch_size, self.window_size, x.size()[1])
        x, _  = self.lstm_3(x)
        x = self.dropout(x)
        
        x = x.reshape(x.size()[-1], batch_size, self.window_size) #(feature_size, batch, window_len)
        x = self.batch_norm3(x.unsqueeze(0))
        
        x = x.view(batch_size, self.window_size, x.size()[1])
        x = x[:, -1, :] # equivalent to return sequence = False on keras :)
        
        x = self.dropout(x)
        
        x = self.linear1(x)
        x = self.activation(x)
        
#         x = self.dropout(x)
        
#         x = self.linear2(x)
#         x = self.activation(x)
#         x = self.dropout(x)
            
        output = self.output_layer(x)
        
        #output = F.log_softmax(output, dim = 1)
        #output = F.softmax(output)
        
        return output
    
    def training_step(self, batch, batch_nb):
        x, y = batch
        output = self(x)
        #loss = F.nll_loss(output, y)
        loss = self.cross_entropy_loss(output, y)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        
        acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
        self.log('train_acc', acc, on_epoch=True, prog_bar=True)
        
        f1 = self.f1_score(torch.max(output, dim=1)[1], y)
        self.log('train_f1', f1, on_epoch=True, prog_bar=True)
        
        return loss
    
    def validation_step(self, batch, batch_nb):
        x, y = batch
        output = self(x)
        #loss = F.nll_loss(output, y)
        loss = self.cross_entropy_loss(output, y)
        self.log('val_loss', loss, on_epoch=True, reduce_fx=torch.mean, prog_bar=True)
        
        #print(torch.max(output, dim=1)[1])
        acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
        self.log('val_acc', acc, on_epoch=True, reduce_fx=torch.mean, prog_bar=True)
        
        f1 = self.f1_score(torch.max(output, dim=1)[1], y)
        self.log('val_f1', f1, on_epoch=True, reduce_fx=torch.mean, prog_bar=True)
        
    def test_step(self, batch, batch_nb):
        x, y = batch
        output = self(x)
        
        #print(y, torch.max(output, dim=1)[1])
        #print(output)
        print(F.softmax(output)) # mantıken fark etmiyor
        print(y, torch.max(output, dim=1)[1])
        
        #loss = F.nll_loss(output, y)
        loss = self.cross_entropy_loss(output, y)
        self.log('test_loss', loss, on_epoch=True, reduce_fx=torch.mean)
        
        acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
        self.log('test_acc', acc, on_epoch=True, reduce_fx=torch.mean)
        
        f1 = self.f1_score(torch.max(output, dim=1)[1], y)
        self.log('test_f1', f1, on_epoch=True, reduce_fx=torch.mean)
        
    def configure_optimizers(self):
        
        optimizer = torch.optim.Adam(model.parameters(), lr= 1e-3, weight_decay=1e-6)#AdamW does weight decay
#         optimizer = torch.optim.AdamW(model.parameters(), lr= 1e-3)#AdamW does weight decay
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

        return [optimizer], [{"scheduler": scheduler}]
    
    def train_dataloader(self):
        return self.train_dl

    def val_dataloader(self):
        return self.val_dl

    def test_dataloader(self):
        return self.test_dl

!rm -rf ../output/models/lstm_model_logs/lstm_v1/version_*

In [17]:
MODEL_NAME = f"{WINDOW_SIZE}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

NameError: name 'FUTURE_PERIOD_PREDICT' is not defined

In [18]:
#logger = WandbLogger(name='lstm.v4',project='pytorchlightning')
logger = TensorBoardLogger("../output/models/lstm_model_logs", name="lstmv3")

In [22]:
model = LSTM_based_classification_model(batch_size=128,)
trainer = pl.Trainer(gpus=-1, 
                     max_epochs= 150,
                     logger = logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [23]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name               | Type             | Params
---------------------------------------------------------
0  | lstm_1             | LSTM             | 71.7 K
1  | batch_norm1        | BatchNorm2d      | 256   
2  | lstm_2             | LSTM             | 132 K 
3  | batch_norm2        | BatchNorm2d      | 256   
4  | lstm_3             | LSTM             | 132 K 
5  | batch_norm3        | BatchNorm2d      | 256   
6  | dropout            | Dropout          | 0     
7  | linear1            | Linear           | 8.3 K 
8  | activation         | ReLU             | 0     
9  | output_layer       | Linear           | 130   
10 | cross_entropy_loss | CrossEntropyLoss | 0     
11 | f1_score           | F1               | 0     
12 | accuracy_score     | Accuracy         | 0     
---------------------------------------------------------
345 K     Trainable params
0         Non-trainable params
345 K     Total params
1.380     Total estimated model 

1

In [ ]:
trainer.test()

In [ ]:
test = TimeSeriesDataset(X[0], y[0], "test")
labels = [test[i][1].item() for i in range(test.__len__())]

In [ ]:
pd.DataFrame({"label": labels}).groupby("label").size()

In [ ]:
183 / (183+131)